In [ ]:
import matplotlib.pyplot as plt
import qiskit
from qiskit.algorithms import VQE
import qiskit_nature
import qiskit_nature.problems.second_quantization
import qiskit_nature.drivers.second_quantization
import qiskit_nature.transformers.second_quantization.electronic
import qiskit_nature.algorithms
from qiskit_nature.drivers import Molecule
import numpy as np
from qiskit.circuit.library import EfficientSU2


def get_qubit_op(molecule,remove_orbitals):
    driver = qiskit_nature.drivers.second_quantization.ElectronicStructureMoleculeDriver(
        molecule=molecule,
        basis='sto3g',
        driver_type=qiskit_nature.drivers.second_quantization.ElectronicStructureDriverType.PYSCF)

    # Define Problem, Use freeze core approximation, remove orbitals.
    problem = qiskit_nature.problems.second_quantization.ElectronicStructureProblem(
        driver,
        remove_orbitals)

    second_q_ops = problem.second_q_ops()  # Get 2nd Quant OP
    num_spin_orbitals = problem.num_spin_orbitals
    num_particles = problem.num_particles

    mapper = qiskit_nature.mappers.second_quantization.ParityMapper()  # Set Mapper
    hamiltonian = second_q_ops['ElectronicEnergy']  # Set Hamiltonian
    # Do two qubit reduction
    converter = qiskit_nature.converters.second_quantization.QubitConverter(mapper,two_qubit_reduction=True)
    reducer = qiskit.opflow.TwoQubitReduction(num_particles)
    qubit_op = converter.convert(hamiltonian)
    qubit_op = reducer.convert(qubit_op)

    return qubit_op, num_particles, num_spin_orbitals, problem, converter

def exact_solver(problem, converter):
    solver = qiskit_nature.algorithms.NumPyMinimumEigensolverFactory()
    calc = qiskit_nature.algorithms.GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result


from qiskit.providers.fake_provider import FakeManila
from qiskit.providers.aer.noise import NoiseModel
from qiskit import Aer

backend = Aer.get_backend('aer_simulator')
device = FakeManila()
NOISE_MODEL = NoiseModel.from_backend(device)
coupling_map = device.configuration().coupling_map
qi = qiskit.utils.QuantumInstance(backend, coupling_map=coupling_map, noise_model=NOISE_MODEL)

distances = np.loadtxt('distances.txt')
distances = distances * 0.529177249 # energy conversion to angstroms
ref_ener_gs = - np.loadtxt('exact_ener.txt')

cobyla = qiskit.algorithms.optimizers.COBYLA(maxiter=300)
spsa = qiskit.algorithms.optimizers.SPSA(maxiter=300)

np_energies=[]
vqe_energies_cobyla = []
vqe_energies_spsa = []
vqe_energies_two_local_cobyla = []
vqe_energies_two_local_spsa=[]

counts = []
values = []

import timeit

start = timeit.default_timer()


for dist in distances:
    # Define Molecule
    molecule = Molecule(
        # Coordinates in Angstrom
        geometry=[
            ["H", [0.0, 0.0, 0.0] ],
            ["H", [dist, 0.0, 0.0] ]
        ],
        multiplicity=1,  # = 2*spin + 1
        charge=0
    )
    (qubit_op, num_particles, num_spin_orbitals,
                             problem, converter) = get_qubit_op(molecule, None)
    
    result = exact_solver(problem,converter)
    np_energies.append(result.total_energies[0].real)
    
    init_state = qiskit_nature.circuit.library.HartreeFock(num_spin_orbitals, num_particles, converter)
   
    uccsd = qiskit_nature.circuit.library.UCCSD(converter,
                     num_particles,
                     num_spin_orbitals,
                     initial_state=init_state)
    
    two_local = EfficientSU2(num_qubits=num_spin_orbitals, entanglement='linear', initial_state=init_state)

    vqe_cobyla = VQE(uccsd, cobyla ,quantum_instance=backend)
    vqe_spsa = VQE(uccsd, spsa ,quantum_instance=backend)
    
    vqe_two_local_cobyla = VQE(two_local, cobyla ,quantum_instance=backend)
    vqe_two_local_spsa = VQE(two_local, spsa ,quantum_instance=backend)
    
    cobyla_calc = vqe_cobyla.compute_minimum_eigenvalue(qubit_op)
    spsa_calc = vqe_spsa.compute_minimum_eigenvalue(qubit_op)
    
    two_local_cobyla_cal = vqe_two_local_cobyla.compute_minimum_eigenvalue(qubit_op)
    two_local_spsa_cal = vqe_two_local_spsa.compute_minimum_eigenvalue(qubit_op)
    
    vqe_result_cobyla = problem.interpret(cobyla_calc).total_energies[0].real
    vqe_result_spsa = problem.interpret(spsa_calc).total_energies[0].real
    
    vqe_result_two_local_cobyla = problem.interpret(two_local_cobyla_cal).total_energies[0].real
    vqe_result_two_local_spsa = problem.interpret(two_local_spsa_cal).total_energies[0].real
        
    vqe_energies_cobyla.append(vqe_result_cobyla)   
    vqe_energies_spsa.append(vqe_result_spsa)   
    vqe_energies_two_local_cobyla.append(vqe_result_two_local_cobyla)   
    vqe_energies_two_local_spsa.append(vqe_result_two_local_spsa)   
    
    print(f"Interatomic Distance: {np.round(dist, 2)}")

print("All energies have been calculated")

stop = timeit.default_timer()

execution_time = stop - start

print("Program Executed in "+str(execution_time)) # It returns time in seconds


h2_noisy_sims = {}
h2_noisy_sims['vqe_energies_cobyla'] = vqe_energies_cobyla
h2_noisy_sims['vqe_energies_spsa'] = vqe_energies_spsa
h2_noisy_sims['vqe_energies_two_local_cobyla'] = vqe_energies_two_local_cobyla
h2_noisy_sims['vqe_energies_two_local_spsa'] = vqe_energies_two_local_spsa
h2_noisy_sims['np_energies'] = np_energies

np.savez("h2_noisy_sims", **h2_noisy_sims)

In [ ]:
import matplotlib.pyplot as plt
import qiskit
from qiskit.algorithms import VQE
import qiskit_nature
import qiskit_nature.problems.second_quantization
import qiskit_nature.drivers.second_quantization
import qiskit_nature.transformers.second_quantization.electronic
import qiskit_nature.algorithms
from qiskit_nature.drivers import Molecule
import numpy as np
from qiskit.primitives import Estimator
from pyscf import scf, gto
from qiskit.circuit.library import EfficientSU2

estimator  = Estimator()

def get_qubit_op(molecule,remove_orbitals):
    driver = qiskit_nature.drivers.second_quantization.ElectronicStructureMoleculeDriver(
        molecule=molecule,
        basis='sto3g',
        driver_type=qiskit_nature.drivers.second_quantization.ElectronicStructureDriverType.PYSCF)

    # Define Problem, Use freeze core approximation, remove orbitals.
    problem = qiskit_nature.problems.second_quantization.ElectronicStructureProblem(
        driver,
        remove_orbitals)

    second_q_ops = problem.second_q_ops()  # Get 2nd Quant OP
    num_spin_orbitals = problem.num_spin_orbitals
    num_particles = problem.num_particles

    mapper = qiskit_nature.mappers.second_quantization.ParityMapper()  # Set Mapper
    hamiltonian = second_q_ops['ElectronicEnergy']  # Set Hamiltonian
    # Do two qubit reduction
    converter = qiskit_nature.converters.second_quantization.QubitConverter(mapper,two_qubit_reduction=True)
    reducer = qiskit.opflow.TwoQubitReduction(num_particles)
    qubit_op = converter.convert(hamiltonian)
    qubit_op = reducer.convert(qubit_op)

    return qubit_op, num_particles, num_spin_orbitals, problem, converter

def exact_solver(problem, converter):
    solver = qiskit_nature.algorithms.NumPyMinimumEigensolverFactory()
    calc = qiskit_nature.algorithms.GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

def hf_ener(d):
    
    mol = gto.M(
      verbose=0,
      atom=[['H',(0, 0, 0)], 
            ['H',(0, 0, d)]],
      basis='sto3g',
      charge = 0
    )
    
    mf = scf.RHF(mol).x2c().run()
    hf = mf.kernel()

    return hf

from qiskit.providers.fake_provider import FakeManila
from qiskit.providers.aer.noise import NoiseModel
from qiskit import Aer, transpile


backend = Aer.get_backend('aer_simulator')
device = FakeManila()
NOISE_MODEL = NoiseModel.from_backend(device)
coupling_map = device.configuration().coupling_map
qi = qiskit.utils.QuantumInstance(backend,
                    coupling_map=coupling_map,
                    noise_model=NOISE_MODEL,
                    measurement_error_mitigation_cls=qiskit.utils.mitigation.CompleteMeasFitter)


distances = np.loadtxt('distances.txt')
distances = distances * 0.529177249 #las energías de la ref ya están en angstrom
ref_ener_gs = - np.loadtxt('exact_ener.txt')
cobyla = qiskit.algorithms.optimizers.COBYLA(maxiter=300)
spsa = qiskit.algorithms.optimizers.SPSA(maxiter=300)

np_energies=[]
vqe_energies_cobyla = []
vqe_energies_spsa = []
vqe_energies_two_local_cobyla = []
vqe_energies_two_local_spsa=[]

import timeit

start = timeit.default_timer()

for dist in distances:
    # Define Molecule
    molecule = Molecule(
        # Coordinates in Angstrom
        geometry=[
            ["H", [0.0, 0.0, 0.0] ],
            ["H", [dist, 0.0, 0.0] ]
        ],
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )
    (qubit_op, num_particles, num_spin_orbitals,
                             problem, converter) = get_qubit_op(molecule, None)
    
    result = exact_solver(problem,converter)
    np_energies.append(result.total_energies[0].real)
    
    init_state = qiskit_nature.circuit.library.HartreeFock(num_spin_orbitals, num_particles, converter)
   
    uccsd = qiskit_nature.circuit.library.UCCSD(converter,
                     num_particles,
                     num_spin_orbitals,
                     initial_state=init_state)
    
    two_local = EfficientSU2(num_qubits=num_spin_orbitals, entanglement='linear', initial_state=init_state)

    vqe_cobyla = VQE(uccsd, cobyla ,quantum_instance=qi)
    vqe_spsa = VQE(uccsd, spsa ,quantum_instance=qi)
    
    vqe_two_local_cobyla = VQE(two_local, cobyla ,quantum_instance=qi)
    vqe_two_local_spsa = VQE(two_local, spsa ,quantum_instance=qi)
    
    cobyla_calc = vqe_cobyla.compute_minimum_eigenvalue(qubit_op)
    spsa_calc = vqe_spsa.compute_minimum_eigenvalue(qubit_op)
    
    two_local_cobyla_cal = vqe_two_local_cobyla.compute_minimum_eigenvalue(qubit_op)
    two_local_spsa_cal = vqe_two_local_spsa.compute_minimum_eigenvalue(qubit_op)
    
    vqe_result_cobyla = problem.interpret(cobyla_calc).total_energies[0].real
    vqe_result_spsa = problem.interpret(spsa_calc).total_energies[0].real
    
    vqe_result_two_local_cobyla = problem.interpret(two_local_cobyla_cal).total_energies[0].real
    vqe_result_two_local_spsa = problem.interpret(two_local_spsa_cal).total_energies[0].real
        
    vqe_energies_cobyla.append(vqe_result_cobyla)   
    vqe_energies_spsa.append(vqe_result_spsa)   
    vqe_energies_two_local_cobyla.append(vqe_result_two_local_cobyla)   
    vqe_energies_two_local_spsa.append(vqe_result_two_local_spsa)   
    
    print(f"Interatomic Distance: {np.round(dist, 2)}")

print("All energies have been calculated")

stop = timeit.default_timer()

execution_time = stop - start

print("Program Executed in "+str(execution_time)) # It returns time in seconds


h2_noisy_sims_EM = {}
h2_noisy_sims_EM['vqe_energies_cobyla'] = vqe_energies_cobyla
h2_noisy_sims_EM['vqe_energies_spsa'] = vqe_energies_spsa
h2_noisy_sims_EM['vqe_energies_two_local_cobyla'] = vqe_energies_two_local_cobyla
h2_noisy_sims_EM['vqe_energies_two_local_spsa'] = vqe_energies_two_local_spsa
h2_noisy_sims_EM['np_energies'] = np_energies

np.savez("h2_ansatz_noisy_EM", **h2_noisy_sims_EM)